# AME end-to-end example (genriesz)

This notebook demonstrates how to estimate an **Average Marginal Effect (AME)**,
i.e., an **average derivative** of the outcome regression function.

We simulate

\[
Y = \sin(X_0) + 0.5 X_1^2 + \varepsilon,
\]

so the true AME for coordinate 0 is

\[
\mathbb{E}[\partial_{x_0} \gamma(X)] = \mathbb{E}[\cos(X_0)].
\]

If \(X_0 \sim N(0,1)\), then \(\mathbb{E}[\cos(X_0)] = \exp(-1/2)\).


In [1]:
import numpy as np
from genriesz import grr_ame, SquaredGenerator, PolynomialBasis

rng = np.random.default_rng(0)


## Synthetic data with known true AME

In [2]:
n = 4000
d = 3

X = rng.normal(size=(n, d))
eps = rng.normal(scale=1.0, size=n)

Y = np.sin(X[:, 0]) + 0.5 * (X[:, 1] ** 2) + eps

true_ame0 = float(np.exp(-0.5))  # E[cos(N(0,1))]
print("Approx. true AME for coordinate 0:", true_ame0)


Approx. true AME for coordinate 0: 0.6065306597126334


## Fit GRR for AME

In [3]:
# A simple polynomial basis on X
basis = PolynomialBasis(degree=3, include_bias=True)

gen = SquaredGenerator(C=0.0).as_generator()

res = grr_ame(
    X=X,
    Y=Y,
    coordinate=0,
    basis=basis,
    generator=gen,
    cross_fit=True,
    folds=5,
    random_state=0,
    estimators=("ra", "rw", "arw", "tmle"),
    outcome_models="shared",
    riesz_penalty="l2",
    riesz_lam=1e-3,
    max_iter=300,
    tol=1e-8,
)

print(res.summary_text())


AME(coord=0) estimates (n=4000)
alpha=0.05 | null=0.0

Estimator         Estimate            SE                           CI     p-value
---------------------------------------------------------------------------------
RA                0.568023    0.00659669       [ 0.555094,  0.580952]           0
RW                0.569997     0.0255014       [ 0.520015,  0.619979]           0
ARW               0.568096     0.0169592       [ 0.534856,  0.601335]           0
TMLE              0.568094     0.0169599       [ 0.534853,  0.601334]           0


## Generator / regularization sweep (SQ / UKL / BP)

Below we compare:

- **SQ-Riesz** (squared generator),
- **UKL-Riesz** (unnormalized KL generator),
- **BP-Riesz** (Basu's power generator, with multiple `omega` values),

under multiple regularization norms (`l2`, `l1`, `l_p`) and multiple regularization strengths.

We report **RA / RW / ARW / TMLE** side-by-side, and also show the estimation error
relative to the Monte Carlo "truth" computed above.

In [7]:
from genriesz import UKLGenerator, BPGenerator

# A small grid over generators and regularization.
generator_grid = [
    ("SQ", SquaredGenerator(C=0.0).as_generator()),
    ("UKL (C=0)", UKLGenerator(C=0.0).as_generator()),
    ("BP (omega=0.1, C=0)", BPGenerator(C=0.0, omega=0.1).as_generator()),
    ("BP (omega=0.2, C=0)", BPGenerator(C=0.0, omega=0.2).as_generator()),
    ("BP (omega=0.5, C=0)", BPGenerator(C=0.0, omega=0.5).as_generator()),
]

penalty_grid = [
    {"penalty": "l2", "lam": 1e-4},
    {"penalty": "l2", "lam": 1e-3},
    {"penalty": "l1", "lam": 1e-4},
    {"penalty": "l1", "lam": 1e-3},
]

rows = []
for gname, gen_i in generator_grid:
    for cfg in penalty_grid:
        res_i = grr_ame(
            X=X,
            Y=Y,
            coordinate=0,
            basis=basis,
            generator=gen_i,
            cross_fit=True,
            folds=3,               # smaller folds for the sweep
            random_state=0,
            estimators=("ra", "rw", "arw", "tmle"),
            outcome_models="shared",
            outcome_link="identity",  # Y is unbounded, so Gaussian TMLE is appropriate
            riesz_penalty=cfg["penalty"],
            riesz_lam=cfg["lam"],
            max_iter=250,
            tol=1e-8,
        )

        row = {
            "generator": gname,
            "penalty": cfg["penalty"],
            "lam": cfg["lam"],
        }

        for k in ("ra", "rw", "arw", "tmle"):
            e = res_i.estimates[k]
            row[f"{k}"] = e.estimate
            row[f"{k}_se"] = e.se
            row[f"{k}_err"] = e.estimate - true_ame0

        rows.append(row)

import pandas as pd

df = pd.DataFrame(rows)
# Sort by absolute ARW error (ARW is typically stable)
df = df.sort_values(by="arw_err", key=lambda s: np.abs(s))
display(df)

generator penalty     lam       ra     ra_se    ra_err  \
17  BP (omega=0.5, C=0)      l2  0.0010  0.56645  0.006685 -0.040081   
10  BP (omega=0.1, C=0)      l1  0.0001  0.56645  0.006685 -0.040081   
18  BP (omega=0.5, C=0)      l1  0.0001  0.56645  0.006685 -0.040081   
8   BP (omega=0.1, C=0)      l2  0.0001  0.56645  0.006685 -0.040081   
9   BP (omega=0.1, C=0)      l2  0.0010  0.56645  0.006685 -0.040081   
16  BP (omega=0.5, C=0)      l2  0.0001  0.56645  0.006685 -0.040081   
5             UKL (C=0)      l2  0.0010  0.56645  0.006685 -0.040081   
6             UKL (C=0)      l1  0.0001  0.56645  0.006685 -0.040081   
19  BP (omega=0.5, C=0)      l1  0.0010  0.56645  0.006685 -0.040081   
12  BP (omega=0.2, C=0)      l2  0.0001  0.56645  0.006685 -0.040081   
13  BP (omega=0.2, C=0)      l2  0.0010  0.56645  0.006685 -0.040081   
14  BP (omega=0.2, C=0)      l1  0.0001  0.56645  0.006685 -0.040081   
4             UKL (C=0)      l2  0.0001  0.56645  0.006685 -0.040081   
11  BP (omega=0.1, C=0)      l1  0.0010  0.56645  0.006685 -0.040081   
15  BP (omega=0.2, C=0)      l1  0.0010  0.56645  0.006685 -0.040081   
7             UKL (C=0)      l1  0.0010  0.56645  0.006685 -0.040081   
1                    SQ      l2  0.0010  0.56645  0.006685 -0.040081   
0                    SQ      l2  0.0001  0.56645  0.006685 -0.040081   
2                    SQ      l1  0.0001  0.56645  0.006685 -0.040081   
3                    SQ      l1  0.0010  0.56645  0.006685 -0.040081   

          rw     rw_se    rw_err       arw    arw_se   arw_err      tmle  \
17  0.595288  0.025486 -0.011243  0.583562  0.019426 -0.022969  0.569381   
10  0.658497  0.023136  0.051966  0.582554  0.018365 -0.023976  0.568362   
18  0.587849  0.025482 -0.018682  0.582152  0.019404 -0.024379  0.569124   
8   0.660513  0.023135  0.053982  0.580962  0.018364 -0.025569  0.568178   
9   0.660512  0.023135  0.053982  0.580962  0.018364 -0.025569  0.568178   
16  0.588427  0.025519 -0.018103  0.579793  0.019415 -0.026738  0.568735   
5   0.578919  0.027191 -0.027611  0.578271  0.019641 -0.028260  0.568613   
6   0.578065  0.027179 -0.028466  0.578176  0.019640 -0.028355  0.568595   
19  0.591563  0.025495 -0.014967  0.577708  0.019398 -0.028823  0.568375   
12  0.677778  0.024179  0.071247  0.577092  0.018921 -0.029439  0.568038   
13  0.677776  0.024178  0.071246  0.577092  0.018921 -0.029439  0.568038   
14  0.677731  0.024174  0.071201  0.577089  0.018919 -0.029442  0.568037   
4   0.575639  0.027207 -0.030892  0.576981  0.019636 -0.029550  0.568375   
11  0.644585  0.023178  0.038054  0.576927  0.018325 -0.029603  0.567680   
15  0.677059  0.024139  0.070529  0.576287  0.018898 -0.030244  0.567908   
7   0.574841  0.027141 -0.031690  0.575717  0.019627 -0.030813  0.568142   
1   0.575361  0.026199 -0.031170  0.569663  0.017072 -0.036868  0.569537   
0   0.576791  0.026184 -0.029739  0.569646  0.017092 -0.036885  0.569517   
2   0.576779  0.026169 -0.029752  0.569638  0.017091 -0.036893  0.569510   
3   0.575307  0.026053 -0.031223  0.569633  0.017068 -0.036898  0.569511   

     tmle_se  tmle_err  
17  0.019458 -0.037150  
10  0.018381 -0.038169  
18  0.019433 -0.037406  
8   0.018378 -0.038353  
9   0.018378 -0.038353  
16  0.019440 -0.037796  
5   0.019639 -0.037917  
6   0.019637 -0.037935  
19  0.019419 -0.038156  
12  0.018932 -0.038493  
13  0.018932 -0.038493  
14  0.018930 -0.038494  
4   0.019634 -0.038156  
11  0.018334 -0.038850  
15  0.018907 -0.038623  
7   0.019625 -0.038389  
1   0.017102 -0.036993  
0   0.017121 -0.037014  
2   0.017121 -0.037021  
3   0.017097 -0.037019